<a href="https://colab.research.google.com/github/Abdullah-Al-Arafat/CAP-5610-Machine-Learning/blob/master/ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CAP 5610: Machine Learning** 
Home work 2


In [1]:
import os
import numpy as np
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad, Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Activation, Flatten, BatchNormalization
from keras import regularizers

Using TensorFlow backend.


Preparation/preprocessing of train, test, and validation dataset 

In [2]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

## Creating validation set from training dataset

indices = np.random.permutation(len(train_images))

val_indices = indices[0:10000]
train_indices = indices[10000:]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]


## Normalizing the Train, test and validation sets

X_train = (Train_images/255).astype('float32')
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255).astype('float32')
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255).astype('float32')
y_test = np_utils.to_categorical(test_labels, 10)

170500096/170498071 [==============================] - 13s 0us/step


**Model without Dropout layers and Data Augmentation** 

In [3]:
model1 = Sequential()
model1.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))


model1.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
#model1.add(Dropout(0.5))
 
model1.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())
#model.add(BatchNormalization())
#model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01))
model1.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model1.add(Dense(64, activation= 'relu'))
model1.add(Dense(10, activation='softmax'))

model1.summary()



Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 48)        13872     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        27712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv

In [4]:
# Initiate Adagrad/RMSprop optimizer
model1.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

## Train the model and get the training and validation loss
history = model1.fit(X_train, y_train, batch_size= 40, epochs= 120,
                    verbose=1, validation_data=(X_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/120
40000/40000 [==============================] - 36s 888us/step - loss: 1.6810 - acc: 0.4214 - val_loss: 1.3328 - val_acc: 0.5488
Epoch 2/120
40000/40000 [==============================] - 28s 700us/step - loss: 1.1302 - acc: 0.6289 - val_loss: 1.0462 - val_acc: 0.6605
Epoch 3/120
40000/40000 [==============================] - 28s 698us/step - loss: 0.9215 - acc: 0.7050 - val_loss: 1.0109 - val_acc: 0.6757
Epoch 4/120
40000/40000 [==============================] - 28s 710us/step - loss: 0.7846 - acc: 0.7535 - val_loss: 0.9052 - val_acc: 0.7071
Epoch 5/120
40000/40000 [==============================] - 30s 742us/step - loss: 0.6789 - acc: 0.7945 - val_loss: 0.8910 - val_acc: 0.7209
Epoch 6/120
40000/40000 [==============================] - 30s 746us/step - loss: 0.5925 - acc: 0.8263 - val_loss: 0.9016 - val_acc: 0.7284
Epoch 7/120
40000/40000 [==============================] - 29s

KeyboardInterrupt: ignored

In [5]:
## Score trained model.
scores = model1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 243us/step
Test loss: 1.9841494254112244
Test accuracy: 0.7339


model1 is clearly overfitting the training dataset, althought I have added L1 and L2 regularization in the Conv layers and FC layers.

**Model2** with dropout layers

In [6]:
model2 = Sequential()
model2.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))


model2.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model1.add(Dropout(0.5))
 
model2.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model2.add(Dropout(0.25))

model2.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))

model2.add(Flatten())
model2.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model2.add(Dense(64, activation= 'relu'))
model2.add(Dense(10, activation='softmax'))

model2.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 48)        13872     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        27712     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
___________________________

In [7]:
# Initiate Adagrad/RMSprop optimizer
model2.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

## Train the model and get the training and validation loss
history = model2.fit(X_train, y_train, batch_size= 40, epochs= 30,
                    verbose=1, validation_data=(X_val, y_val))

Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 34s 843us/step - loss: 1.7304 - acc: 0.3987 - val_loss: 1.3951 - val_acc: 0.5277
Epoch 2/30
40000/40000 [==============================] - 34s 847us/step - loss: 1.3044 - acc: 0.5622 - val_loss: 1.1723 - val_acc: 0.6130
Epoch 3/30
40000/40000 [==============================] - 33s 834us/step - loss: 1.1258 - acc: 0.6323 - val_loss: 1.0529 - val_acc: 0.6586
Epoch 4/30
40000/40000 [==============================] - 32s 800us/step - loss: 1.0166 - acc: 0.6769 - val_loss: 1.0235 - val_acc: 0.6735
Epoch 5/30
40000/40000 [==============================] - 33s 836us/step - loss: 0.9403 - acc: 0.7009 - val_loss: 0.9532 - val_acc: 0.7014
Epoch 6/30
40000/40000 [==============================] - 34s 853us/step - loss: 0.8791 - acc: 0.7241 - val_loss: 0.9150 - val_acc: 0.7137
Epoch 7/30
40000/40000 [==============================] - 33s 826us/step - loss: 0.8319 - acc: 0.7417 - val_loss: 0.

In [8]:
## Score trained model.
scores = model2.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 230us/step
Test loss: 0.8814269060134887
Test accuracy: 0.7711


**Model2** significantly reduce the varince, now I'll introduce data augmentation in the model3

In [0]:
#data augmentation

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=60,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')

In [0]:
model3 = Sequential()
model3.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))


model3.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model3.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model3.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
 
model3.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model3.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model3.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model3.add(Flatten())
model3.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model3.add(Dense(64, activation= 'relu'))
model3.add(Dense(10, activation='softmax'))

In [11]:
model3.compile(loss='categorical_crossentropy', 
               optimizer='adagrad', metrics=['accuracy'])



## Train the model and get the training and validation loss
model3.fit_generator(datagen.flow(X_train, y_train, batch_size=40),
                    steps_per_epoch=train_images.shape[0]/ 40,epochs= 120,
                    verbose=1,validation_data=(X_val,y_val))

Epoch 1/120
1250/1250 [==============================] - 57s 45ms/step - loss: 1.8879 - acc: 0.3326 - val_loss: 1.6427 - val_acc: 0.4284
Epoch 2/120
1250/1250 [==============================] - 56s 45ms/step - loss: 1.6012 - acc: 0.4417 - val_loss: 1.5006 - val_acc: 0.4749
Epoch 3/120
1250/1250 [==============================] - 56s 45ms/step - loss: 1.4708 - acc: 0.4979 - val_loss: 1.3240 - val_acc: 0.5484
Epoch 4/120
1250/1250 [==============================] - 59s 47ms/step - loss: 1.3746 - acc: 0.5299 - val_loss: 1.3202 - val_acc: 0.5565
Epoch 5/120
1250/1250 [==============================] - 61s 49ms/step - loss: 1.3076 - acc: 0.5555 - val_loss: 1.1718 - val_acc: 0.6089
Epoch 6/120
1250/1250 [==============================] - 58s 47ms/step - loss: 1.2557 - acc: 0.5780 - val_loss: 1.1702 - val_acc: 0.6122
Epoch 7/120
1250/1250 [==============================] - 60s 48ms/step - loss: 1.2165 - acc: 0.5891 - val_loss: 1.1410 - val_acc: 0.6256
Epoch 8/120
1250/1250 [==================

In [12]:
## Score trained model.
scores = model3.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 228us/step
Test loss: 0.809547947216034
Test accuracy: 0.7635


**Moel4**  Dropout with Data augmentation

In [0]:
model4 = Sequential()
model4.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.1))


model4.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.25))

model4.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model4.add(Dropout(0.5))
 
model4.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model4.add(Dropout(0.25))

model4.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.1))

model4.add(Flatten())
model4.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model4.add(Dense(64, activation= 'relu'))
model4.add(Dense(10, activation='softmax'))

In [17]:
model4.compile(loss='categorical_crossentropy', 
               optimizer='adagrad', metrics=['accuracy'])



## Train the model and get the training and validation loss
model4.fit_generator(datagen.flow(X_train, y_train, batch_size=40),
                    steps_per_epoch=train_images.shape[0]/ 40,epochs= 120,
                    verbose=1,validation_data=(X_val,y_val))

Epoch 1/120
1250/1250 [==============================] - 52s 42ms/step - loss: 2.0188 - acc: 0.2863 - val_loss: 1.7681 - val_acc: 0.3966
Epoch 2/120
1250/1250 [==============================] - 49s 39ms/step - loss: 1.7864 - acc: 0.3767 - val_loss: 1.6368 - val_acc: 0.4357
Epoch 3/120
1250/1250 [==============================] - 48s 39ms/step - loss: 1.6795 - acc: 0.4195 - val_loss: 1.5158 - val_acc: 0.4893
Epoch 4/120
1250/1250 [==============================] - 49s 39ms/step - loss: 1.6070 - acc: 0.4438 - val_loss: 1.4721 - val_acc: 0.4980
Epoch 5/120
1250/1250 [==============================] - 49s 39ms/step - loss: 1.5546 - acc: 0.4660 - val_loss: 1.3900 - val_acc: 0.5249
Epoch 6/120
1250/1250 [==============================] - 49s 39ms/step - loss: 1.5097 - acc: 0.4840 - val_loss: 1.3863 - val_acc: 0.5308
Epoch 7/120
1250/1250 [==============================] - 50s 40ms/step - loss: 1.4816 - acc: 0.4947 - val_loss: 1.4001 - val_acc: 0.5272
Epoch 8/120
1250/1250 [==================

For k-fold validation, I have used the best model from above

In [15]:
## Get the training dataset again

from sklearn.model_selection import KFold

X_train = train_images.reshape(train_images.shape[0], 32, 32, 3)
y_train_Kfold = np_utils.to_categorical(train_labels, 10)
X_train_Kfold = (X_train/255).astype('float32')


scores = []
cv = KFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X_train_Kfold):
  
    ## K-fold Split of the dataset(where K=20, 19 of the folds are for training and 1 of them is for validating)
    X_train, X_val, y_train, y_val = X_train_Kfold[train_index], X_train_Kfold[test_index], y_train_Kfold[train_index], y_train_Kfold[test_index]

    ## Compile the optimizer and train 
    model2.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    model2.fit(X_train, y_train)

    ## Get the score on the test dataset
    scores.append(model2.evaluate(X_test, y_test))
    
print("K-fold cross validation DONE...")
print("The Cross validation Score [Accuracy]",np.mean(scores[1]))

Epoch 1/1
10000/10000 [==============================] - 3s 305us/step
Epoch 1/1
10000/10000 [==============================] - 2s 221us/step
Epoch 1/1
10000/10000 [==============================] - 2s 231us/step
Epoch 1/1
10000/10000 [==============================] - 2s 236us/step
Epoch 1/1
10000/10000 [==============================] - 3s 250us/step
K-fold cross validation DONE...
The Cross validation Score [Accuracy] 0.8608836936950683
